<a href="https://colab.research.google.com/github/surendra-crax/120Days-DataScience/blob/main/Day9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploads = files.upload()

Saving titanic.csv to titanic.csv


# STEP:1- Loading dataset and inspecting structure

In [70]:
import pandas as pd


df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [71]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [72]:
df.shape

(891, 12)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [74]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


# STEP:2 - Standardizing column names.

In [83]:
df.columns =df.columns.str.replace("_","")

In [84]:
df.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'sex', 'age', 'sibsp',
       'parch', 'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')

#STEP:3 - Missing value cleaning

In [86]:
#cleaning age
df["age"] = df["age"].fillna(
    df.groupby("pclass")["age"].transform("median")
)

In [87]:
#cleaning cabin
df["cabin"] = df["cabin"].fillna("Unknown")

In [89]:
mode_value = df["embarked"].mode()[0]
df["embarked"] = df["embarked"].fillna(mode_value)

In [90]:
df.isnull().sum()

,0
passengerid,0
survived,0
pclass,0
name,0
sex,0
age,0
sibsp,0
parch,0
ticket,0
fare,0


# STEP:4 - Duplicate Handling

In [91]:
#this scans all rows and give no of duplicates
df.duplicated().sum()


np.int64(0)

In [92]:
#removing duplicates
df = df.drop_duplicates().copy()


In [93]:
df.duplicated().sum()

np.int64(0)

# STEP:5 - Fixing Data Types

In [94]:
df.dtypes


,0
passengerid,int64
survived,int64
pclass,int64
name,object
sex,object
age,float64
sibsp,int64
parch,int64
ticket,object
fare,float64


In [95]:
df[["age","fare"]]=df[["age","fare"]].astype(int)

In [96]:
df.dtypes

,0
passengerid,int64
survived,int64
pclass,int64
name,object
sex,object
age,int64
sibsp,int64
parch,int64
ticket,object
fare,int64


In [97]:
df[["sex","ticket","cabin"]] = df[["sex","ticket","cabin"]].astype("category")

In [98]:
df.dtypes

,0
passengerid,int64
survived,int64
pclass,int64
name,object
sex,category
age,int64
sibsp,int64
parch,int64
ticket,category
fare,int64


# STEP:6 - Outlier detection and implementation of IQR(Inter quartile Method)

In [100]:
Q1 = df["fare"].quantile(0.25)      #25th percentile -> Q1
Q3 = df["fare"].quantile(0.75)      #75th percentile -> Q2
print(Q1, Q3)


7.0 31.0


In [101]:
# calculating IQR
IQR = Q3 - Q1
IQR

np.float64(24.0)

In [102]:
#calculating lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(lower_bound,upper_bound)

-29.0 67.0


In [104]:
#counting outliers
df[(df["fare"]<lower_bound)| (df["fare"]>upper_bound)].shape

(114, 12)

In [106]:
#remove outliers
df = df[(df["fare"] >= lower_bound)& (df["fare"]<= upper_bound)].copy()

In [107]:
#Reset index
df = df.reset_index(drop=True)

In [108]:
df["fare"].describe()

,fare
count,777.000000
mean,17.530245
std,13.819135
min,0.000000
25%,7.000000
50%,13.000000
75%,26.000000
max,66.000000


In [109]:
#Exporting cleaned dataset
df.to_csv("titanic_cleaned.csv", index=False)
